**GROUP PROJECT**

**Visualization 1 - Muhammad Syahmi**

For Visualization 1, we will present **Total of movies Released according to Genre** in Pie Chart form.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

1. Read the csv file 

In [3]:
df = pd.read_csv('netflix_titles.csv')
df.head()

FileNotFoundError: [Errno 2] File netflix_titles.csv does not exist: 'netflix_titles.csv'

2. Removing rows which is not movie.

In [ ]:
df.drop(df.loc[df['type']!='Movie'].index, inplace=True)
df

3. Removing all rows with null value and removing string from "duration" column and only leaving number

In [ ]:
df.dropna(inplace= True)
df = df.assign(duration = lambda x: x['duration'].str.extract('(\d+)'))
df.head()

4. Removing days and years, leaving on month in column "date_added"

In [ ]:
month = df['date_added'].str.split(pat = ' [1-9]',n = 1, expand = True)
df['date_added'] = month[0]
df['duration'] = pd.to_numeric(df['duration'])
df.head()

5. Split the data in column "listed_in" which represent genre

In [ ]:
genresplit = df['listed_in'].str.split(pat = ",", expand = True)
genresplit[3] = genresplit[0]
a = genresplit.groupby([0]).count()
genresplit

6. Plot the graf of **Total of movies Released according to Genre**

In [ ]:
fig , axes = plt.subplots(figsize =(20,20))
month = a.index
font = {'family': 'serif',
        'color':  'white',
        'weight': 'normal',
        'size': 24,
        }
myexplode = [0,0,0,0,0,0,0,0,0,0.1,0,0,0,0,0,0,0]
box = plt.pie(a[3], startangle = 90,shadow = True)
plt.legend(month, loc = 3,)
plt.title("Total of movies Released according to Genre",  fontdict = font);

In [ ]:
fig , axes = plt.subplots(figsize =(20,20))
a[1].plot.bar()

font = {'family': 'serif',
        'color':  'white',
        'weight': 'normal',
        'size': 24,
        }
plt.title("Total of movies Released according to Genre",  fontdict = font);

**Visualization 2 - Syarifah**

For Visualization 2, we will present **Rating based on sum of Top 3 Country** in Bar Chart form.

1. Call the **df** dataframe from visualization 1

In [ ]:
df

2. Drop out all the rows that contain NaN values

In [ ]:
cleaned = df.dropna()
cleaned

3. Group by the data by rating and country, and calculate the sum of country in each rating

In [ ]:
countrysplit = cleaned['country'].str.split(pat = ",", expand = True)
countrysplit['rating'] = df['rating']
a = countrysplit.groupby(['rating' , 0]).count()
a["sum"] = a.sum(axis=1)
a

4. Drop the columns **1,2,3,4,5,6** which represent the count and only take the column **sum**

In [ ]:
a = a.drop(columns=[1,2,3,4,5,6])
a 

5. Sort the sum in ascending order and only take the top 3 country for each rating

In [ ]:
b = a.groupby(level=[0,1]).sum().reset_index().sort_values(['rating', 'sum'], ascending=[1,0]).groupby('rating').head(3)
b

6. Rename the column 0 to country

In [ ]:
b.rename(columns = { 0 :'country'}, inplace = True)
b

7. Plot the bar graph of **Rating based on sum of Top 3 Country**

In [ ]:
g= b.groupby(['rating'])[['country','sum']].sum()
g.plot.bar(title="Rating based on sum of Top 3 country", color="darkolivegreen");

**Visualization 3 - Hilman**

For Visualization 3, we will present **Average Duration of Each Genre** in Bar Chart form.

1. Call the **df** dataframe from visualization 1

In [ ]:
df

2. Keeping the first column of genre and rename into "Genre"

In [ ]:
genresplit = df['listed_in'].str.split(",",n=3, expand = True)
genresplit = genresplit.rename(columns={0: 'Genre', 1: 'no1', 2:'no2'})
genresplit = genresplit.drop(columns=['no1','no2'])
genresplit['duration'] = df['duration']
genresplit['Genre'] = genresplit['Genre'].astype('str') 
genresplit[['duration']] = genresplit[['duration']].apply(pd.to_numeric)

genresplit.dtypes

3. Displaying the average duration based on genre

In [ ]:
avgdura = genresplit
avgdura = avgdura.drop_duplicates(keep='last')
avgdura.groupby('Genre').mean().reset_index()
avgdura = genresplit.groupby('Genre').mean()
avgdura = avgdura.rename(columns={'duration': 'Average Duration of Each Genre'})
avgdura['Average Duration of Each Genre'] = avgdura['Average Duration of Each Genre'].apply(lambda x: round(x, 2))
avgdura.reset_index(level=0, inplace=True)

avgdura

4. Plot the bar graph of **Average Duration of Each Genre**

In [ ]:
avgdura.plot.bar(x= 'Genre', y = 'Average Duration of Each Genre', color='purple', figsize=(10,10));

Machine Learning

In [ ]:
import pandas as pd
import numpy as np
import json
import nltk
import re
import csv
import matplotlib.pyplot as plt 
from os import path
from PIL import Image
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

movies = pd.DataFrame()
movies['Title'] = df['title']
movies['Genre'] = genresplit['Genre']
movies['Description'] = df['description']
movies

In [ ]:
text = 